In [ ]:
import cv2 as cv
import numpy as np

def do_canny(frame):
    gray = cv.cvtColor(frame, cv.COLOR_RGB2GRAY)
    blur = cv.GaussianBlur(gray, (5, 5), 0)
    canny = cv.Canny(blur, 50, 150)
    return canny

def region_of_interest(frame):
    height = frame.shape[0]
    polygons = np.array([
                            [(200, height), (1100, height), (550, 250)]
                        ])
    mask = np.zeros_like(frame)
    cv.fillPoly(mask, polygons, 255)
    segment = cv.bitwise_and(frame, mask)
    return segment

def average_slope_intercept(frame, lines):
    left = []
    right = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope = parameters[0]
        y_intercept = parameters[1]
        if slope < 0:
            left.append((slope, y_intercept))
        else:
            right.append((slope, y_intercept))
    left_avg = np.average(left, axis = 0)
    right_avg = np.average(right, axis = 0)
    left_line = make_coordinates(frame, left_avg)
    right_line = make_coordinates(frame, right_avg)
    return np.array([left_line, right_line])

def make_coordinates(frame, parameters):
    slope , intercept = parameters
    y1 = frame.shape[0]
    y2 = int(y1 - 250)
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    return np.array([x1, y1, x2, y2])

def display_lines(frame, lines):
    lines_visualize = np.zeros_like(frame)
    if lines is not None:
        for x1, y1, x2, y2 in lines:
            cv.line(lines_visualize, (x1, y1), (x2, y2), (0, 255, 0), 5)
    return lines_visualize

cap = cv.VideoCapture("lane_data/video.mp4")
while (cap.isOpened()):
    ret, frame = cap.read()
    canny = do_canny(frame)
    cv.imshow("canny", canny)
    segment = region_of_interest(canny)
    hough = cv.HoughLinesP(segment, 2, np.pi / 180, 100, np.array([]), minLineLength = 100, maxLineGap = 50)
    lines = average_slope_intercept(frame, hough)
    lines_visualize = display_lines(frame, lines)
    cv.imshow("hough", lines_visualize)
    output = cv.addWeighted(frame, 0.9, lines_visualize, 1, 1)
    cv.imshow("output", output)
    if cv.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

In [ ]:
최근에는 자율주행차라고도 불리는 자율주행차가 지속 가능하고 안전하며 편리하며 혼잡 없는 교통성을 제공함으로써 도시 이동성에 혁명을 일으킬 수 있는 잠재력을 가지고 있다. 사람이 거의 또는 전혀 입력하지 않고 환경을 감지하고 안전하게 이동할 수 있는 차량입니다. 인공지능(AI)의 응용으로서 이 차량 자율성은 신호등, 표지판, 불분명한 차선 표시, 보행자 등과 같은 여러 가지 과제를 안고 있다. 이 프로젝트의 목표는 자율 주행 모드에서 자율주행 자동차를 시뮬레이션하는 것이다. 이 프로젝트는 컴퓨터 비전 및 딥 러닝 기술의 도움을 받아 차선 감지, 교통 신호 분류, 신호 분류, 차량 및 보행자 감지로 구성된다.
